# Logistic Regression

Logistic regression is a statistical model and classification technique used to predict the probability of a binary outcome, typically coded as 0 or 1 (e.g., yes/no, success/failure, true/false). Unlike linear regression, which is used to predict continuous numerical values, logistic regression is designed specifically for categorical outcomes. It's a type of generalized linear model (GLM).

Logistic regression deals with binary outcomes, making it suitable for classification problems. The outcome variable (dependent variable) is a categorical variable that takes two values, often represented as 0 and 1.

This type of regression models the relationship between the independent variables and the **log odds** of the binary outcome. The log odds (**logit**) is a linear combination of the predictor variables. The **logistic function**, also known as the **sigmoid function**, is used to transform the log-odds into probabilities. The sigmoid function takes any real number and maps it to a value between 0 and 1 that represents the probability of the event occurring.

$$P(Y=1)=\frac{1}{1 + e^{-(\beta_0 + \beta_1x_1 + \beta_2x_2 + \ldots + \beta_nx_k)}},$$

where $P(Y=1)$ is the probability of a *success*; $\beta_0, \beta_1, \beta_2, \ldots, \beta_n$ are the coefficients to be estimated; and $x_1, x_2,\ldots, x_n$ are the independent variables.

Coefficients in logistic regression can be interpreted in terms of how they affect the natural logarithm of the odds of the binary outcome. For a one-unit change in an independent variable, the coefficient represents how much the natural logarithm of the odds changes.

Logistic regression is widely used in fields as diverse as medicine, biology, social sciences, marketing, and machine learning for tasks such as predicting disease outcomes, customer churn, credit risk, spam detection, and more. It is a valuable tool for binary classification and is often a fundamental building block of more complex machine learning algorithms.

Before we go any further, we need to load the libraries that we will be using.

In [ ]:
import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import seaborn                 as sns
import statsmodels.api         as sm

Once again, we will work with the `diabetes dataset`. This time we will use the logistic regression model to predict whether a person has diabetes, that is, the *success* or the positive case is someone with this disease.

In [ ]:
data = pd.read_csv('diabetes-dataset.csv')
data.head()

## Unbalanced Sets

In machine learning, unbalanced sets refer to datasets where the number of examples in different classes or categories is significantly different. This means that one class has a much larger number of instances (samples) than another class. For example, in a binary classification problem, if class A has 90% of the examples and class B has only 10%, this is considered an unbalanced dataset.

Unbalanced sets are quite common in many real-world applications, including:

- **Fraud Detection:** Genuine transactions are much more frequent than fraudulent ones.
- **Medical Diagnostics:** Rare diseases have fewer cases compared to common ones.
- **Anomaly Detection:** Anomalies or rare events are typically infrequent.
- **Text Classification:** Categories like "spam" are often less common than "non-spam."
- **Image Recognition:** Rare objects or events may occur infrequently.

In this case, the `diabetes-dataset` is an example of an unbalanced set: about 66% of the observations belong to the negative class (people without diabetes), while about a 34% are observations of the positive class (people with diabetes).

In [ ]:
data['Outcome'].value_counts()

Well, the data is not very unbalanced, but it is quite unbalanced. Ideally, we would like to have roughly the same number of observations for each category, but as mentioned earlier, there are applications where this is not possible. When this is the case, there are some techniques that help us deal with the issues of having unbalanced categories, which we will not discuss here, but it is important to mention it because it will affect the performance of our model, as we will see later.

## Training and Testing Sets

When we develop a model, we do not use all of our data for training, what we do is that we divide the data we have into two sets: the **training set** and the **testing set**. A general rule of thumb is to use 80% of the data for training and 20% for testing our model. There are variations of this depending on the available amount of data, but in this case it is a good starting point. By the way, all examples of our training data should be chosen randomly to avoid any bias; it is not a good practice to choose these examples in a deterministic way.

In [ ]:
np.random.seed(1337) 
number_of_rows = data.shape[0]
index_train = np.random.choice(range(number_of_rows), int(0.8 * number_of_rows), replace=False)
index_test = np.asarray(list(set(range(number_of_rows)) - set(index_train)))
train_set = data.iloc[index_train] 
test_set = data.iloc[index_test] 
print(train_set.shape)
print(test_set.shape)

In [ ]:
y_train = train_set['Outcome']
train_set = train_set.drop(columns='Outcome')
y_test = test_set['Outcome']
test_set = test_set.drop(columns='Outcome')

In [ ]:
y_train

In [ ]:
train_set

## Training the Model

We will use the `Logit` method from the `statsmodel` library. As expected, we will train the model only with the training data.

In [ ]:
logistic_model = sm.Logit(y_train, train_set).fit()
logistic_model.summary()

As you can see, the logistic regression summary provides a lot of information. In particular, we get the confidence intervals for each parameter of the mode. In addition, the `Logit` method also performs a two-tailed hypothesis test on each parameter: it checks whether a given parameter is equal to zero. In this case, there are several variables that are considered irrelevant for predicting whether a person has diabetes or not: assuming a significance level of $\alpha=0.05$, the parameters of `SkinThickness`, `Insulin`, `BMI` and `DiabetesPedigreeFunction` are very likely equal to zero. Note that in this case the normal distribution is used to obtain these intervals and to perform the hypothesis tests

For more information on the calculation of these quantities, please refer to the presentation and the bibliography on the last slide of this document.

## Predictions

Once trained, the model is ready to make predictions. Note that the predictions are made on the test set: this is important because we want to contrast the model's predictions with the true labels of the data. This will be used later to evaluate the model.

To compute the predictions, we use the `predict` method associated with our instance of the `sm.Logit` object. 

In [ ]:
y_pred = logistic_model.predict(test_set)
y_pred

Note that the predictions are numbers between zero and one. This is so because these numbers are actually probabilities: the ith probability can be interpreted as the probability that the variable $x_i$ belongs to the positive class.

Now we need to turn these probabilities into labels. The typical way to do this is to label a given observation $x_i$ as one if the ith probability is greater than or equal to 0.5, and if it is not, then it is classified as an element of the negative category. The threshold of 0.5 is the typical choice, but it can be modified.

In [ ]:
threshold = 0.5
y_pred[y_pred < threshold] = 0
y_pred[y_pred >= threshold] = 1
y_pred

## Evaluating the Model

So we built the Naive Bayes classifier and we trained it, but is it good? To know how good our model is, we need **evaluation metrics**. There are tons of metrics, and the ideal metric or metrics will have to be chosen depending on what is important for your particular application. For now, we will mention some of the most common ones, but before we go any further, we need to say a few things about the **confusion matrix**.

#### Confusion Matrix

A confusion matrix is a table that allows us to visualize the performance of a classification algorithm. 

<img src="confusion.png" alt="Drawing" style="width: 700px;"/>

This type of table is called this because it allows us to observe whether an algorithm mislabels two classes (Image taken from https://en.wikipedia.org/wiki/Precision_and_recall). 

#### Accuracy

Accuracy is defined as follows:

$$
\text{Accuracy}=\frac{\text{true positives} + \text{true negatives}}{\text{true positives} + \text{false positives} +  \text{true negatives} + \text{false negatives}}.
$$

This metric is useful when both classes are equally important and when we have balanced set, which is not quite the case in this application.

#### Precision

The ratio of correctly labeled positive cases to all examples classified as positive is called **precision**:

$$
\text{Precision}=\frac{\text{true positives}}{\text{true positives} + \text{false positives}}.
$$

If we are interested in reducing the number of false positives and we have unbalanced sets, precision is a good choice as an evaluation metric. In this application, a false positive would be a person who does not have diabetes being diagnosed with the disease. As mentioned earlier, the dataset is unbalanced: the majority of persons do not have diabetes.

#### Recall

Recall is the ratio of examples correctly identified as positive cases to all true positive examples in our data. This metric can be thought of as the sensitivity of our model:

$$
\text{Recall}=\frac{\text{true positives}}{\text{true positives} + \text{false negatives}}.
$$

If we want to pay special attention to the false negatives that our model detects, and if our sets are unbalanced, then this can be one of our performance metrics. Suppose we want to build a model that detects a dangerous disease. In this case, we are not interested in telling a person that they do not have the disease when they do (false negative). 

#### F1 Score

The F1 score is equal to the harmonic mean of precision and recall. It is useful when we want to have a balance between precision and recall and when we do not have a balanced dataset. It is defined as 

$$
\text{F1 Score}=2\frac{\text{Precision} * \text{Recall}}{\text{Precision} + \text{Recall}}.
$$

The following function computes these evaluation metrics.

In [ ]:
def performance_metrics(results):
    
    positives = results[['observed', 'prediction']][results['observed'] == 1]
    negatives = results[['observed', 'prediction']][results['observed'] == 0]
    
    true_negatives = negatives[negatives['observed'] == negatives['prediction']].shape[0]
    false_positives = negatives[negatives['observed'] != negatives['prediction']].shape[0]
    true_positives = positives[positives['observed'] == positives['prediction']].shape[0]
    false_negatives = positives[positives['observed'] != positives['prediction']].shape[0]
    
    confusion_matrix = {'actual positives' : [true_positives, false_negatives], 
                        'actual negatives' : [false_positives, true_negatives]}
    
    confusion_matrix_df = pd.DataFrame.from_dict(confusion_matrix, orient='index', 
                                                 columns=['predicted positives', 'predicted negatives'])
    
    accuracy = (true_positives + true_negatives) / (true_positives + false_positives +  true_negatives + false_negatives)
    precission = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precission * recall) / (precission + recall)
    
    metrics = {'Accuracy' : accuracy, 'Precission' : precission, 'Recall' : recall, 'F1 Score' : f1_score}
    
    metrics_df = pd.DataFrame.from_dict(metrics, orient='index', columns=['Metrics'])
    
    return confusion_matrix_df, metrics_df   

The above function receives a dataframe that contains the true labels and the predictions of the model. This said, we proceed to create such a dataframe in the next cell.

In [ ]:
labels = {'observed': y_test, 'prediction': y_pred}
labels_df = pd.DataFrame(labels)
labels_df.head()

And we are ready for some results.

In [ ]:
confusion_matrix, metrics = performance_metrics(labels_df)
confusion_matrix

In [ ]:
metrics

As you can see, the performance of the model is not very good. Given this, your mission is to make some modifications to the above procedure in order to improve the results that were obtained. 